# 2.1 开始构建新人答疑机器人

## 🚄 前言 
因为你们公司规章制度是私有知识，所以大模型并不能直接回答关于你们公司规章制度的问题。RAG（Retrieval-Augmented Generation，检索增强式生成）是目前最常见的大模型私有领域知识问答应用构建方案，最适合用于开发你所需要的新人答疑机器人。

但从头构建一个完善的 RAG 应用，需要你关注很多细节：你需要解析各种格式的文档、对文档建立索引、开发检索逻辑、调用大模型等等。

借助 LlamaIndex，你可以更快地构建 RAG 应用，就像你可以借助 Spring、Flask、Express 等框架来更快地构建 web 应用一样。

## 🍁 课程目标

学完本课程后，你将能够：<br>
- 掌握直接向通义千问模型提问的方法
- 学会通过 LlamaIndex 框架快速创建RAG应用

## 📖 课程目录
本章节作为 RAG 应用构建的起点，我们将从最简单大模型 API 调用开始，逐步构建一个 RAG 应用：
- 1.&nbsp;计算环境准备    
- 2.&nbsp;直接向通义千问提问    
- 3.&nbsp;实现 RAG 问答机器人    


## 💻 1. 计算环境准备
### 1.1. 安装依赖
在运行后续课程中的代码之前，你需要先通过下面的命令安装依赖。由于目前你正在使用 Notebook ，你可以直接点击代码块旁边的运行按钮来运行代码或命令。

In [ ]:
! pip install -r requirements.txt -i https://mirrors.aliyun.com/pypi/simple/

### 1.2. 获取并配置 API Key 用于调用大模型
为了调用大模型，你需要前往阿里云大模型服务平台百炼，[开通模型调用服务](https://bailian.console.aliyun.com/#/model-market)并[创建一个 API Key](https://bailian.console.aliyun.com/?apiKey=1#/api-key)。

> 如果页面顶部显示如下，则表示你尚未开通百炼的模型调用服务。
> 
> <img src="https://help-static-aliyun-doc.aliyuncs.com/assets/img/zh-CN/5298748271/p856749.png" width="600">

完成 API Key 的创建后，你可以运行下面的代码，在弹出的输入框中输入你的 API Key，即可完成环境变量的配置。

In [8]:
import os

from config.load_key import load_key
load_key()

# 生产环境中请勿将 API Key 输出到日志中，避免泄露
print(f'''你配置的 API Key 是：{os.environ["DASHSCOPE_API_KEY"][:5]+"*"*5}''')

你配置的 API Key 是：sk-1a*****


## 💬 2. 直接向通义千问提问
完成上述配置后，你就可以通过下面的代码来调用通义千问大模型，同时也可以验证前面准备的环境是否正常。

<!-- > 如果你在运行下边代码时遇到了问题，可以参考[首次调用通义千问API](https://help.aliyun.com/zh/model-studio/getting-started/first-api-call-to-qwen#f92b9b9cc7huw) 中的**账号设置**部分。 -->

你可以问一个关于你们公司的问题，比如：**我们公司项目管理应该用什么工具**

In [9]:
from llama_index.llms.dashscope import DashScope
from llama_index.core.base.llms.types import MessageRole, ChatMessage

# 定义 llm 用于后续和大模型交互
llm = DashScope(model_name="qwen-plus")
# 定义输入到大模型中的messages，你可以在此定义system message与user message。system message用于指定大模型的人设、回复方式等，user message用于接收用户的输入内容。
messages = [
  ChatMessage(role=MessageRole.SYSTEM, content="你负责教育内容开发公司的答疑，你的名字叫公司小蜜，你要回答学员的问题。"),
  ChatMessage(role=MessageRole.USER, content="我们公司项目管理应该用什么工具")
]

# notebook 中运行代码块后，会打印最后一个变量，无需使用 print 方法
llm.chat(messages).message.content

'选择项目管理工具时，需要考虑团队的具体需求、项目的特性以及预算等因素。以下是一些常用的项目管理工具，您可以根据自己的实际情况来选择：\n\n1. **Trello**：适合小型团队和简单项目管理。它以看板的形式展示任务，非常直观，易于上手。\n\n2. **Jira**：对于软件开发团队来说，Jira 是一个非常强大的工具，支持敏捷开发方法，如Scrum和Kanban。它提供了丰富的功能，包括问题跟踪、时间跟踪等。\n\n3. **Asana**：适用于各种规模的团队，提供任务分配、进度跟踪等功能。Asana 的界面友好，支持多种视图（列表、看板、日历等），方便不同类型的项目管理。\n\n4. **Microsoft Project**：对于大型企业或复杂项目，Microsoft Project 提供了详细的计划制定、资源分配和成本控制等功能。它更适合那些需要严格控制项目进度和成本的场景。\n\n5. **Monday.com**：这是一个高度可定制化的平台，可以根据团队的需求创建不同的工作流程。它支持团队协作、文件共享等功能，非常适合创意团队使用。\n\n6. **Wrike**：Wrike 也是一个全面的项目管理工具，支持项目规划'

可以看到大模型在回答问题时耗时比较长，如果你希望更快的看到回答，可以使用stream_chat方法进行流式输出，大模型将一段一段地返回结果，以便让你构建的机器人有更好的用户体验。

In [10]:
responses = llm.stream_chat(messages)
for response in responses:
    print(response.delta, end="")

选择项目管理工具时，需要考虑团队的具体需求、项目的特性以及预算等因素。以下是一些常用的项目管理工具，您可以根据自己的实际情况来选择：

1. **Trello**：适合小型团队和简单项目管理。它以看板的形式展示任务，非常直观，易于上手。

2. **Jira**：对于软件开发团队来说，Jira 是一个非常强大的工具，支持敏捷开发方法，如Scrum和Kanban。它提供了丰富的功能，包括问题跟踪、时间跟踪等。

3. **Asana**：适用于各种规模的团队，提供任务分配、进度跟踪等功能。Asana 的界面友好，支持多种视图（列表、看板、日历等），便于团队成员了解项目进展。

4. **Microsoft Project**：对于大型企业或复杂项目，Microsoft Project 提供了详细的计划制定、资源分配和成本控制等功能。它更适合那些需要严格控制项目时间和成本的场景。

5. **Monday.com**：这是一个高度可定制化的平台，可以根据不同的业务需求创建工作流程。它支持团队协作、文件共享等功能，非常适合跨部门合作的项目。

6. **Wrike**：Wrike 也是一个全面的项目管理解决方案，支持

大模型虽然做出了回答，但只是列举了市面上比较常见的工具。

为了让大模型可以参考公司的知识库进行回答，你可以创建一个RAG应用，这样大模型就可以参考公司的知识信息来进行回复了。

## 📚 3. 实现RAG问答机器人

### 基本流程

我们从前边的例子可以看到，尽管大模型对这个问题：”我们公司项目管理应该用什么工具“做出了回答，但是由于大模型并不了解公司规章制度，所以回答的不够准确。

为了让大模型能回答关于公司规章制度的问题，你准备构建一个 RAG 应用，RAG应用的工作流程包括：
1. 解析：加载公司规章制度文档（如pdf、docx等），并解析为文本形式；
2. 分段：对解析后的文档进行分段，因为大模型的输入长度是有限的；
3. 建立索引：使用嵌入模型（embedding 模型）对切片后的文档建立索引，并以向量数据库形式存储，便于后续检索；
4. 发起提问：用户输入的问题与向量数据库的知识进行匹配，并通过大模型生成结合知识库与用户问题的回复。

<img src="https://img.alicdn.com/imgextra/i1/O1CN01GjUPUs1dIbaxdDowf_!!6000000003713-2-tps-1971-1221.png" width="600" />

通过LlamaIndex，只需要几行代码就可以完成上述功能。

In [11]:
# 导入依赖
from llama_index.embeddings.dashscope import DashScopeEmbedding,DashScopeTextEmbeddingModels
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex
# 这两行代码是用于消除 WARNING 警告信息，避免干扰阅读学习，生产环境中建议根据需要来设置日志级别
import logging
logging.basicConfig(level=logging.ERROR)

print("正在解析文件...")
# LlamaIndex提供了SimpleDirectoryReader方法，可以直接将指定文件夹中的文件加载为document对象，对应着解析过程
documents = SimpleDirectoryReader('./docs').load_data()

print("正在创建索引...")
# from_documents方法包含切片与建立索引步骤
index = VectorStoreIndex.from_documents(
    documents,
    # 指定embedding 模型
    embed_model=DashScopeEmbedding(
        # 你也可以使用阿里云提供的其它embedding模型：https://help.aliyun.com/zh/model-studio/getting-started/models#3383780daf8hw
        model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V2
    ))
print("正在创建提问引擎...")
query_engine = index.as_query_engine(
    # 设置为流式输出
    streaming=True,
    # 此处使用qwen-plus模型，你也可以使用阿里云提供的其它qwen的文本生成模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
    llm=DashScope(model_name="qwen-plus")
    )
print("正在生成回复...")
streaming_response = query_engine.query('我们公司项目管理应该用什么工具')
print("回答是：")
streaming_response.print_response_stream()

正在解析文件...
正在创建索引...
正在创建提问引擎...
正在生成回复...
回答是：
公司项目管理可以使用如Jira或Trello这样的项目管理软件。这些工具能够帮助团队更好地规划和跟踪项目的进度，确保任务按时完成。同时，它们还支持任务分配、时间跟踪和团队协作等功能，有助于提高工作效率。

这段代码已经可以让大模型参考你们公司的知识库，并精确回答了Jira和Trello这两个你们公司正在使用的项目管理软件。

<img src="https://img.alicdn.com/imgextra/i4/O1CN01Jk00aE23pkGNTssf2_!!6000000007305-0-tps-832-646.jpg" alt="PAI DSW Notebook Demo" width="300px">

### 保存与加载索引
你可能会发现，创建索引消耗的时间比较长。如果能够将索引保存到本地，并在需要使用的时候直接加载，而不是重新建立索引，那就可以大幅减从问题到给出回复的耗时。

你可以借助LlamaIndex提供的保存与加载索引文件的方法来实现这一点。

In [12]:
# 将索引保存为本地文件
index.storage_context.persist("knowledge_base/test")
print("索引文件保存到了knowledge_base/test")

索引文件保存到了knowledge_base/test


In [13]:
# 将本地索引文件加载为索引
from llama_index.core import StorageContext,load_index_from_storage
storage_context = StorageContext.from_defaults(persist_dir="knowledge_base/test")
index = load_index_from_storage(storage_context,embed_model=DashScopeEmbedding(
        model_name=DashScopeTextEmbeddingModels.TEXT_EMBEDDING_V2
    ))
print("成功从knowledge_base/test路径加载索引")

成功从knowledge_base/test路径加载索引


从本地加载索引后，我们再次进行提问：

In [14]:
print("正在创建提问引擎...")
query_engine = index.as_query_engine(
    # 设置为流式输出
    streaming=True,
    # 此处使用qwen-plus模型，你也可以使用阿里云提供的其它qwen的文本生成模型：https://help.aliyun.com/zh/model-studio/getting-started/models#9f8890ce29g5u
    llm=DashScope(model_name="qwen-plus")
    )
print("正在生成回复...")
streaming_response = query_engine.query('我们公司项目管理应该用什么工具')
print("回答是：")
streaming_response.print_response_stream()

正在创建提问引擎...
正在生成回复...
回答是：
公司项目管理可以使用如Jira或Trello这样的项目管理软件。这些工具能够帮助团队更好地规划和跟踪项目的进度，确保任务按时完成。同时，它们也支持团队成员之间的高效协作，便于需求的收集和优先级的确认。

### 封装代码
你可以将上述代码进行封装，以便在后续持续迭代时快速使用。

In [15]:
from chatbot import rag

# 引文在前面的步骤中已经建立了索引，因此这里可以直接加载索引。如果需要重建索引，可以增加一行代码：rag.indexing()
index = rag.load_index(persist_path='./knowledge_base/test')
query_engine = rag.create_query_engine(index=index)

rag.ask('我们公司项目管理应该用什么工具', query_engine=query_engine)

公司项目管理可以使用如Jira或Trello这样的项目管理软件。这些工具能够帮助团队更好地规划和跟踪项目的进度，确保任务按时完成。同时，它们也支持团队成员之间的高效协作，便于需求的收集和优先级的确认。

## 拓展阅读

如果你想要创建以输入框形式交互的RAG应用，请参考：[百炼RAG实践教程文档](https://help.aliyun.com/zh/model-studio/use-cases/build-rag-application-based-on-local-retrieval)。

<img src="https://help-static-aliyun-doc.aliyuncs.com/assets/img/zh-CN/4139126271/p844266.gif" alt="我的notebook" width="600px">

你可以在下载[文档中的压缩包](https://help.aliyun.com/zh/model-studio/use-cases/build-rag-application-based-on-local-retrieval#b91232e354cb4)后，找到chat.py中的get_model_response函数并进行修改，适配到本教程介绍的方法。

## ✅ 本节小结
通过学习本节课程，你已经掌握了LlamaIndex的基本用法，并可以搭建一个基本的答疑机器人。在下一节中，你将学习到如何通过提示词工程来持续优化答疑机器人的回复效果。

## 🔥 课后小测验


【单选题】 以下代码片段的作用是什么？（ ）
```python
index.storage_context.persist(persist_dir=PERSIST_DIR)
```
A. 从磁盘加载嵌入向量。

B. 将嵌入向量存储到内存中。

C. 将嵌入向量持久化到指定的目录。

D. 创建一个新的 StorageContext 对象。

答案：C

## ✉️ 评价反馈
感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。